In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 17:47:07.775446 140443612604224 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 17:47:07.782476 140443612604224 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 17:47:07.788238 140443612604224 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 17:47:07.794351 140443612604224 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 17:47:07.800366 140443612604224 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 17:47:07.808301 140443612604224 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 17:47:07.851291 140443612604224 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 17:47:07.925924 140443612604224 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9280 train_acc= 0.3929 val_loss= 1.9307 val_acc= 0.3933 time= 1.0971
Epoch: 0002 train_loss= 1.9095 train_acc= 0.4143 val_loss= 1.9149 val_acc= 0.4000 time= 0.0227
Epoch: 0003 train_loss= 1.8894 train_acc= 0.4357 val_loss= 1.8982 val_acc= 0.4133 time= 0.0192
Epoch: 0004 train_loss= 1.8687 train_acc= 0.4357 val_loss= 1.8807 val_acc= 0.4133 time= 0.0234
Epoch: 0005 train_loss= 1.8475 train_acc= 0.4286 val_loss= 1.8627 val_acc= 0.4067 time= 0.0190
Epoch: 0006 train_loss= 1.8257 train_acc= 0.4143 val_loss= 1.8441 val_acc= 0.4000 time= 0.0188
Epoch: 0007 train_loss= 1.8043 train_acc= 0.4143 val_loss= 1.8262 val_acc= 0.3767 time= 0.0179
Epoch: 0008 train_loss= 1.7841 train_acc= 0.4071 val_loss= 1.8093 val_acc= 0.3733 time= 0.0178
Epoch: 0009 train_loss= 1.7655 train_acc= 0.4071 val_loss= 1.7940 val_acc= 0.3700 time= 0.0242
Epoch: 0010 train_loss= 1.7486 train_acc= 0.4071 val_loss= 1.7808 val_acc= 0.3700 time= 0.0229
Epoch: 0011 train_loss= 1.7335 train_acc= 0.4071 v

Epoch: 0097 train_loss= 0.6498 train_acc= 0.9214 val_loss= 0.9528 val_acc= 0.7833 time= 0.0182
Epoch: 0098 train_loss= 0.6434 train_acc= 0.9214 val_loss= 0.9488 val_acc= 0.7833 time= 0.0179
Epoch: 0099 train_loss= 0.6370 train_acc= 0.9214 val_loss= 0.9465 val_acc= 0.7867 time= 0.0228
Epoch: 0100 train_loss= 0.6312 train_acc= 0.9214 val_loss= 0.9444 val_acc= 0.7900 time= 0.0191
Epoch: 0101 train_loss= 0.6253 train_acc= 0.9214 val_loss= 0.9409 val_acc= 0.7933 time= 0.0231
Epoch: 0102 train_loss= 0.6188 train_acc= 0.9214 val_loss= 0.9359 val_acc= 0.7967 time= 0.0179
Epoch: 0103 train_loss= 0.6119 train_acc= 0.9286 val_loss= 0.9279 val_acc= 0.7933 time= 0.0189
Epoch: 0104 train_loss= 0.6057 train_acc= 0.9286 val_loss= 0.9212 val_acc= 0.7933 time= 0.0196
Epoch: 0105 train_loss= 0.6003 train_acc= 0.9286 val_loss= 0.9163 val_acc= 0.7900 time= 0.0189
Epoch: 0106 train_loss= 0.5954 train_acc= 0.9357 val_loss= 0.9123 val_acc= 0.7900 time= 0.0187
Epoch: 0107 train_loss= 0.5904 train_acc= 0.9500 v

Epoch: 0185 train_loss= 0.3593 train_acc= 0.9786 val_loss= 0.7508 val_acc= 0.8233 time= 0.0189
Epoch: 0186 train_loss= 0.3575 train_acc= 0.9786 val_loss= 0.7471 val_acc= 0.8233 time= 0.0244
Epoch: 0187 train_loss= 0.3562 train_acc= 0.9714 val_loss= 0.7440 val_acc= 0.8300 time= 0.0237
Epoch: 0188 train_loss= 0.3552 train_acc= 0.9714 val_loss= 0.7434 val_acc= 0.8267 time= 0.0228
Epoch: 0189 train_loss= 0.3547 train_acc= 0.9714 val_loss= 0.7432 val_acc= 0.8167 time= 0.0227
Epoch: 0190 train_loss= 0.3519 train_acc= 0.9714 val_loss= 0.7416 val_acc= 0.8100 time= 0.0229
Epoch: 0191 train_loss= 0.3494 train_acc= 0.9786 val_loss= 0.7406 val_acc= 0.8133 time= 0.0224
Epoch: 0192 train_loss= 0.3482 train_acc= 0.9786 val_loss= 0.7415 val_acc= 0.8133 time= 0.0189
Epoch: 0193 train_loss= 0.3474 train_acc= 0.9786 val_loss= 0.7420 val_acc= 0.8133 time= 0.0236
Epoch: 0194 train_loss= 0.3462 train_acc= 0.9786 val_loss= 0.7422 val_acc= 0.8100 time= 0.0241
Epoch: 0195 train_loss= 0.3447 train_acc= 0.9786 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.7709
accuracy = 0.8200


# Gaussian Graph Convolution

In [15]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
    mean_regularizer=l2(5e-4),
    variance_regularizer=l2(5e-4),
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

W1110 17:47:13.396539 140443612604224 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:28: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 17:47:13.398080 140443612604224 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update

In [16]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1), H1)

W1110 17:47:13.435772 140443612604224 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/losses.py:27: kl_divergence (from tensorflow.python.ops.distributions.kullback_leibler) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [18]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9415 train_acc= 0.4429 val_loss= 1.9434 val_acc= 0.3533 time= 0.4335
Epoch: 0002 train_loss= 1.9366 train_acc= 0.5714 val_loss= 1.9393 val_acc= 0.4700 time= 0.0192
Epoch: 0003 train_loss= 1.9295 train_acc= 0.6357 val_loss= 1.9347 val_acc= 0.5567 time= 0.0236
Epoch: 0004 train_loss= 1.9210 train_acc= 0.7357 val_loss= 1.9296 val_acc= 0.5533 time= 0.0201
Epoch: 0005 train_loss= 1.9112 train_acc= 0.7214 val_loss= 1.9241 val_acc= 0.5567 time= 0.0236
Epoch: 0006 train_loss= 1.9022 train_acc= 0.7143 val_loss= 1.9175 val_acc= 0.5900 time= 0.0185
Epoch: 0007 train_loss= 1.8912 train_acc= 0.7857 val_loss= 1.9114 val_acc= 0.5600 time= 0.0239
Epoch: 0008 train_loss= 1.8808 train_acc= 0.7500 val_loss= 1.9046 val_acc= 0.5933 time= 0.0189
Epoch: 0009 train_loss= 1.8720 train_acc= 0.7214 val_loss= 1.8965 val_acc= 0.5733 time= 0.0201
Epoch: 0010 train_loss= 1.8582 train_acc= 0.7429 val_loss= 1.8899 val_acc= 0.5567 time= 0.0190
Epoch: 0011 train_loss= 1.8458 train_acc= 0.7286 v

Epoch: 0090 train_loss= 0.2704 train_acc= 0.9857 val_loss= 0.6881 val_acc= 0.8167 time= 0.0191
Epoch: 0091 train_loss= 0.2619 train_acc= 0.9857 val_loss= 0.6816 val_acc= 0.8167 time= 0.0192
Epoch: 0092 train_loss= 0.2534 train_acc= 0.9857 val_loss= 0.6757 val_acc= 0.8133 time= 0.0195
Epoch: 0093 train_loss= 0.2468 train_acc= 0.9857 val_loss= 0.6684 val_acc= 0.8133 time= 0.0191
Epoch: 0094 train_loss= 0.2396 train_acc= 0.9857 val_loss= 0.6646 val_acc= 0.8167 time= 0.0185
Epoch: 0095 train_loss= 0.2324 train_acc= 0.9857 val_loss= 0.6591 val_acc= 0.8167 time= 0.0185
Epoch: 0096 train_loss= 0.2256 train_acc= 0.9857 val_loss= 0.6569 val_acc= 0.8200 time= 0.0190
Epoch: 0097 train_loss= 0.2193 train_acc= 0.9857 val_loss= 0.6503 val_acc= 0.8200 time= 0.0192
Epoch: 0098 train_loss= 0.2131 train_acc= 0.9857 val_loss= 0.6472 val_acc= 0.8200 time= 0.0187
Epoch: 0099 train_loss= 0.2064 train_acc= 0.9857 val_loss= 0.6449 val_acc= 0.8200 time= 0.0192
Epoch: 0100 train_loss= 0.2011 train_acc= 0.9857 v

In [19]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.6061
accuracy = 0.8090


# Attacks